!pip install ta-lib
!pip install numba

In [42]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import yfinance
import datetime
import talib
import numba
from numba import njit

In [43]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 2)

In [44]:
btc_price = vbt.YFData.download(
    ['BTC-USD'],
    start = start_date,
    end = end_date,
    missing_index=['drop'],
    interval = '1m'
    ).get('Close')

In [45]:
RSI = vbt.IndicatorFactory.from_talib('RSI')

##### The following way of implementing bactesters is 30%  faster than simply using the VBT

In [46]:
@njit
def produce_signal(rsi, entry, exit):
    trend = np.where(rsi>exit, -1, 0)
    trend = np.where(rsi<entry, 1, trend)
    return trend

def custom_indicator(close, rsi_window=14, entry=30, exit=70):
#     rsi = vbt.RSI.run(close, window = rsi_window).rsi
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    return produce_signal(rsi, entry, exit)

In [47]:
ind = vbt.IndicatorFactory(
        class_name= 'Combination',
        short_name='comb',
        input_names=['close'],
        param_names=['rsi_window','entry','exit'],
        output_names=['value']
        ).from_apply_func(
                custom_indicator,
                rsi_window=14,
                entry=30,
                exit=70
                )

In [48]:
rsi_windows =np.arange(10,40, step=3, dtype=int),
master_returns = []
# use for loop to increase speed and decrease the ram load:
for rsi_window in rsi_windows:
    res = ind.run(
            btc_price,
            rsi_window=rsi_window,
            entry=np.arange(10,40, step=4, dtype=int),
            exit=np.arange(60,80, step=4, dtype=int),
            param_product = True
            )
    entries = res.value == 1.0
    exits = res.value == -1.0
    pf = vbt.Portfolio.from_signals(btc_price,entries,exits)
    master_returns.append(pf.total_return())
returns = pd.concat(master_returns)

In [49]:
print(returns.max())
print(returns.idxmax())

0.028256079987327213
(28, 34, 60)


##### Saving btc_price as csv file and then using it
btc_price.to_csv('data01.csv')

btc_price = pd.read_csv('data01.csv')

btc_price.set_index('Datetime',inplace=True)